In [7]:
import vtk
import pandas as pd
import numpy as np
from vtk.util.numpy_support import numpy_to_vtk

import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderer
)


In [8]:
# Load the NOx data
nox_data = pd.read_csv('../../NOx/B_NOx_0.dat', sep='\s+')

#print(nox_data.head())

#convert numbers as strings to floats 
nox_data['vel'] = pd.to_numeric(nox_data['vel'], errors='coerce')
nox_data['dir'] = pd.to_numeric(nox_data['dir'], errors='coerce')


# Convert the direction from degrees to radians
nox_data['dir_rad'] = np.radians(nox_data['dir'])


In [9]:

# calculating the vectors for velocity
velocity_vectors = np.vstack([nox_data['vel']*np.sin(nox_data['dir']), nox_data['vel']*np.cos(nox_data['dir']), np.zeros_like(nox_data['vel'])]).T

# Create the vtkPoints object and add the coordinates to it
points = vtk.vtkPoints()
for i in range(len(nox_data)):
    points.InsertPoint(i, nox_data['x'][i], nox_data['y'][i], 0)

# Create the vtkPolyData object and add the points to it
nox_polydata = vtk.vtkPolyData()
nox_polydata.SetPoints(points)

# Convert the velocity and direction into a vector
vecs = vtk.vtkDoubleArray()
vecs.SetNumberOfComponents(3)
vecs.SetName("Vectors")


for i in range(len(nox_data)):
    # Insert points
    points.InsertPoint(i, nox_data['x'][i], nox_data['y'][i], 0)

    # Calculate vector components
    x_comp = nox_data['vel'][i] * np.cos(nox_data['dir_rad'][i])
    y_comp = nox_data['vel'][i] * np.sin(nox_data['dir_rad'][i])
    z_comp = 0

    # Insert vectors
    vecs.InsertTuple3(i, x_comp, y_comp, z_comp)

# Create the vtkPolyData object and add the points and vectors to it
nox_polydata.SetPoints(points)
nox_polydata.GetPointData().SetVectors(vecs)


# Convert the concentration into a scalar
concentration = numpy_to_vtk(nox_data['conc'].values)

# Add the concentration to the polydata
nox_polydata.GetPointData().SetScalars(concentration)

1

In [10]:
# Create a cone
cone = vtk.vtkConeSource()

# Create a glyph3D object and set the cone as the source
glyph = vtk.vtkGlyph3D()
glyph.SetSourceConnection(cone.GetOutputPort())
glyph.SetInputData(nox_polydata)
glyph.SetVectorModeToUseVector()
glyph.SetScaleModeToScaleByVector()
glyph.SetColorModeToColorByScalar()
glyph.Update()

# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(glyph.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)


In [11]:
# Initialize the renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

# Create the render window
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(900, 600)
render_window.SetWindowName('Polution Distribution')


# Create the interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Start the visualization
interactor.Initialize()
interactor.Start()
